## Getting the training dataset

In [1]:
!wget -c http://data.vision.ee.ethz.ch/cvl/DIV2K/DIV2K_train_HR.zip

--2025-03-19 06:41:38--  http://data.vision.ee.ethz.ch/cvl/DIV2K/DIV2K_train_HR.zip
Resolving data.vision.ee.ethz.ch (data.vision.ee.ethz.ch)... 129.132.52.178, 2001:67c:10ec:36c2::178
Connecting to data.vision.ee.ethz.ch (data.vision.ee.ethz.ch)|129.132.52.178|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://data.vision.ee.ethz.ch/cvl/DIV2K/DIV2K_train_HR.zip [following]
--2025-03-19 06:41:39--  https://data.vision.ee.ethz.ch/cvl/DIV2K/DIV2K_train_HR.zip
Connecting to data.vision.ee.ethz.ch (data.vision.ee.ethz.ch)|129.132.52.178|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3530603713 (3.3G) [application/zip]
Saving to: ‘DIV2K_train_HR.zip’

DIV2K_train_HR.zip  100%[===================>]   3.29G  20.6MB/s    in 2m 26s  

2025-03-19 06:44:06 (23.0 MB/s) - ‘DIV2K_train_HR.zip’ saved [3530603713/3530603713]



In [2]:
import shutil
import os

zip_file = "/kaggle/working/DIV2K_train_HR.zip"
extract_folder = "/kaggle/working/DIV2K_train_HR"

shutil.unpack_archive(zip_file, extract_folder)
os.remove(zip_file)
print("Unzipped and deleted!")

Unzipped and deleted!


## Adding Noise to images

In [3]:
import os
import numpy as np
import imageio.v2 as imageio  # Use imageio.v2 to avoid deprecation warnings
import torch
import zipfile
import shutil

device = "cuda" if torch.cuda.is_available() else "cpu"

# Paths
HR_DIR = "/kaggle/working/DIV2K_train_HR/DIV2K_train_HR"  # High-resolution images
NOISY_DIR = "/kaggle/working/DIV2K_train_LR_noisy/DIV2K_train_noisy"  # Output directory for noisy images
ZIP_FILE_PATH = "/kaggle/working/noisy_images.zip"  # Path to the zip file containing processed images

# Ensure output directory exists
os.makedirs(NOISY_DIR, exist_ok=True)

import cupy as cp

def add_noise(image, sigma=50):
    image = cp.array(image / 255, dtype=cp.float32)
    noise = cp.random.normal(0, sigma / 255, image.shape)
    gauss_noise = image + noise
    return (gauss_noise * 255).get()  # Convert back to NumPy

def save_image(image, path):
    """
    Saves an image after clipping and rounding to uint8 format.
    image: Image as numpy array.
    path: Save location.
    """
    image = np.round(np.clip(image, 0, 255)).astype(np.uint8)
    imageio.imwrite(path, image)

def crop_image(image, s=8):
    """
    Crops an image so its width & height are multiples of 's'.
    """
    h, w, c = image.shape
    image = image[:h - h % s, :w - w % s, :]
    return image

# Process all images in the HR directory
for img_name in os.listdir(HR_DIR):
    if img_name.endswith(".png"):  # Process only PNG files
        img_path = os.path.join(HR_DIR, img_name)
        noisy_img_path = os.path.join(NOISY_DIR, img_name)

        # Read and process the image
        img = imageio.imread(img_path)
        img = crop_image(img)  # Ensure size is multiple of 8
        img_noise = add_noise(img, sigma=50)  # Add noise

        # Save the noisy image
        save_image(img_noise, noisy_img_path)
        print(f"Processed: {img_name}")

# Create a zip file of all processed images for download
with zipfile.ZipFile(ZIP_FILE_PATH, 'w', zipfile.ZIP_DEFLATED) as zipf:
    for img_name in os.listdir(NOISY_DIR):
        img_path = os.path.join(NOISY_DIR, img_name)
        if img_name.endswith(".png"):  # Only include PNG files in the zip
            zipf.write(img_path, os.path.basename(img_path))

print("All images processed and saved in", NOISY_DIR)
print(f"Processed images have been zipped and can be downloaded from {ZIP_FILE_PATH}")


Processed: 0051.png
Processed: 0604.png
Processed: 0348.png
Processed: 0570.png
Processed: 0599.png
Processed: 0212.png
Processed: 0183.png
Processed: 0422.png
Processed: 0745.png
Processed: 0713.png
Processed: 0322.png
Processed: 0517.png
Processed: 0555.png
Processed: 0195.png
Processed: 0330.png
Processed: 0143.png
Processed: 0055.png
Processed: 0475.png
Processed: 0146.png
Processed: 0544.png
Processed: 0054.png
Processed: 0050.png
Processed: 0302.png
Processed: 0462.png
Processed: 0627.png
Processed: 0217.png
Processed: 0722.png
Processed: 0632.png
Processed: 0010.png
Processed: 0749.png
Processed: 0300.png
Processed: 0172.png
Processed: 0080.png
Processed: 0290.png
Processed: 0374.png
Processed: 0420.png
Processed: 0379.png
Processed: 0596.png
Processed: 0036.png
Processed: 0734.png
Processed: 0431.png
Processed: 0356.png
Processed: 0759.png
Processed: 0378.png
Processed: 0320.png
Processed: 0726.png
Processed: 0792.png
Processed: 0245.png
Processed: 0285.png
Processed: 0097.png


## Defining DataLoaders

In [4]:
import os
import cv2
import torch
import numpy as np
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader
from PIL import Image  # Import PIL
import matplotlib.pyplot as plt

# Dataset class
class DenoiseDataset(Dataset):
    def __init__(self, clean_dir, noisy_dir, transform=None):
        self.clean_images = sorted(os.listdir(clean_dir))
        self.noisy_images = sorted(os.listdir(noisy_dir))
        self.clean_dir = clean_dir
        self.noisy_dir = noisy_dir
        self.transform = transform

    def __len__(self):
        return len(self.clean_images)

    def __getitem__(self, idx):
        clean_path = os.path.join(self.clean_dir, self.clean_images[idx])
        noisy_path = os.path.join(self.noisy_dir, self.noisy_images[idx])

        # Load images using OpenCV (NumPy arrays)
        clean_img = cv2.imread(clean_path)
        noisy_img = cv2.imread(noisy_path)

        # Convert BGR to RGB
        clean_img = cv2.cvtColor(clean_img, cv2.COLOR_BGR2RGB)
        noisy_img = cv2.cvtColor(noisy_img, cv2.COLOR_BGR2RGB)

        # Convert NumPy array to PIL Image
        clean_img = Image.fromarray(clean_img)
        noisy_img = Image.fromarray(noisy_img)

        # Apply transformations
        if self.transform:
            clean_img = self.transform(clean_img)
            noisy_img = self.transform(noisy_img)

        return noisy_img, clean_img

# Data augmentation & transformations
transform = transforms.Compose([
    transforms.Resize((512, 512)),
    transforms.ToTensor()  # Convert to PyTorch tensor
])

# Define dataset paths
train_clean_dir = "/kaggle/working/DIV2K_train_HR/DIV2K_train_HR"
train_noisy_dir = "/kaggle/working/DIV2K_train_LR_noisy/DIV2K_train_noisy"

# Create dataset and dataloader
train_dataset = DenoiseDataset(train_clean_dir, train_noisy_dir, transform=transform)
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)

## Defining UNET model

In [5]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class UNet(nn.Module):
    def __init__(self, in_channels=3, out_channels=3, features=[64, 128, 256, 512]):
        super(UNet, self).__init__()
        
        # Encoder
        self.encoders = nn.ModuleList()
        for feature in features:
            self.encoders.append(self._conv_block(in_channels, feature))
            in_channels = feature
        
        # Bottleneck
        self.bottleneck = self._conv_block(features[-1], features[-1] * 2)
        
        # Decoder
        self.decoders = nn.ModuleList()
        for feature in reversed(features):
            self.decoders.append(nn.ConvTranspose2d(feature * 2, feature, kernel_size=2, stride=2))
            self.decoders.append(self._conv_block(feature * 2, feature))
        
        # Final Output Layer
        self.final_conv = nn.Conv2d(features[0], out_channels, kernel_size=1)

    def _conv_block(self, in_channels, out_channels):
        """Double Convolution Block"""
        return nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
        )

    def forward(self, x):
        skip_connections = []
        for encoder in self.encoders:
            x = encoder(x)
            skip_connections.append(x)
            x = F.max_pool2d(x, kernel_size=2, stride=2)
        
        x = self.bottleneck(x)

        skip_connections = skip_connections[::-1]
        for i in range(0, len(self.decoders), 2):
            x = self.decoders[i](x)  # Upconvolution
            skip_connection = skip_connections[i // 2]

            if x.shape != skip_connection.shape:
                x = F.interpolate(x, size=skip_connection.shape[2:], mode="bilinear", align_corners=True)

            x = torch.cat((skip_connection, x), dim=1)  # Skip connection
            x = self.decoders[i + 1](x)  # Convolution Block

        return self.final_conv(x)

In [6]:
import torch
import torch.optim as optim
import torch.nn as nn
import torchvision.transforms as transforms
from torch.cuda.amp import autocast, GradScaler

# Initialize UNet Model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = UNet(in_channels=3, out_channels=3).to(device)

# Loss Function & Optimizer
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3)
scaler = GradScaler()  # Enable Mixed Precision Training

# Define Transform with Random Cropping
train_transform = transforms.Compose([
    transforms.RandomCrop(128),  # Crop to 128x128 patches
    transforms.ToTensor()
])

# Training Loop
num_epochs = 20
accumulation_steps = 4  # Effective batch size multiplier

for epoch in range(num_epochs):
    model.train()
    epoch_loss = 0
    optimizer.zero_grad()

    for i, (noisy_imgs, clean_imgs) in enumerate(train_loader):
        noisy_imgs, clean_imgs = noisy_imgs.to(device), clean_imgs.to(device)

        with autocast():  # Enable Mixed Precision
            outputs = model(noisy_imgs)
            loss = criterion(outputs, clean_imgs) / accumulation_steps  # Scale loss

        scaler.scale(loss).backward()  # Backpropagate scaled loss

        if (i + 1) % accumulation_steps == 0:  # Update weights every 'accumulation_steps' batches
            scaler.step(optimizer)
            scaler.update()
            optimizer.zero_grad()

        epoch_loss += loss.item() * accumulation_steps  # Reverse scaling

    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss/len(train_loader):.6f}")

<ipython-input-6-1c2907ee7dc3>:14: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = GradScaler()  # Enable Mixed Precision Training
<ipython-input-6-1c2907ee7dc3>:34: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():  # Enable Mixed Precision


Epoch 1/20, Loss: 4.812734
Epoch 2/20, Loss: 0.091043
Epoch 3/20, Loss: 0.017900
Epoch 4/20, Loss: 0.013111
Epoch 5/20, Loss: 0.007509
Epoch 6/20, Loss: 0.005184
Epoch 7/20, Loss: 0.004327
Epoch 8/20, Loss: 0.003890
Epoch 9/20, Loss: 0.003581
Epoch 10/20, Loss: 0.003076
Epoch 11/20, Loss: 0.002662
Epoch 12/20, Loss: 0.002370
Epoch 13/20, Loss: 0.002308
Epoch 14/20, Loss: 0.002150
Epoch 15/20, Loss: 0.002039
Epoch 16/20, Loss: 0.001940
Epoch 17/20, Loss: 0.001852
Epoch 18/20, Loss: 0.001804
Epoch 19/20, Loss: 0.001757
Epoch 20/20, Loss: 0.001732


In [ ]:
# model = UNet(in_channels=3, out_channels=3).to(device)  # Initialize model
# model.load_state_dict(torch.load("unet_model.pth"))  # Load weights into the model
# model.eval() 

In [10]:
from skimage.metrics import peak_signal_noise_ratio as psnr
from skimage.metrics import structural_similarity as ssim


def evaluate_model(model, dataloader, device):
    model.eval()
    total_psnr = 0
    total_ssim = 0
    num_samples = 0

    with torch.no_grad():
        for noisy_imgs, clean_imgs in dataloader:
            noisy_imgs, clean_imgs = noisy_imgs.to(device), clean_imgs.to(device)
            denoised_imgs = model(noisy_imgs)

            for i in range(noisy_imgs.shape[0]):
                clean_np = clean_imgs[i].cpu().numpy().transpose(1, 2, 0)  # (H, W, C)
                denoised_np = denoised_imgs[i].cpu().numpy().transpose(1, 2, 0)

                clean_np = np.clip(clean_np, 0, 1)
                denoised_np = np.clip(denoised_np, 0, 1)

                # Compute PSNR
                img_psnr = psnr(clean_np, denoised_np, data_range=1.0)

                # Compute SSIM with win_size fix
                try:
                    img_ssim = ssim(clean_np, denoised_np, data_range=1.0, win_size=3, channel_axis=-1)
                except ValueError as e:
                    print(f"Skipping SSIM for small image: {clean_np.shape} - {e}")
                    img_ssim = 0  # Assign 0 if SSIM cannot be computed

                total_psnr += img_psnr
                total_ssim += img_ssim
                num_samples += 1

    avg_psnr = total_psnr / num_samples
    avg_ssim = total_ssim / num_samples
    print(f"Validation PSNR: {avg_psnr:.2f} dB, SSIM: {avg_ssim:.4f}")
    return avg_psnr, avg_ssim

## Getting Validation Dataset

In [8]:
!gdown --fuzzy --id 1iYurwSVBUxoN6fQwUGP-UbZkTZkippGx

/usr/local/lib/python3.10/dist-packages/gdown/__main__.py:140: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From (original): https://drive.google.com/uc?id=1iYurwSVBUxoN6fQwUGP-UbZkTZkippGx
From (redirected): https://drive.google.com/uc?id=1iYurwSVBUxoN6fQwUGP-UbZkTZkippGx&confirm=t&uuid=73c0aa0a-c04b-44c2-80c4-5f5fbbf16485
To: /kaggle/working/noise.zip
100%|█████████████████████████████████████████| 811M/811M [00:07<00:00, 103MB/s]


In [9]:
!wget -c http://data.vision.ee.ethz.ch/cvl/DIV2K/DIV2K_valid_HR.zip

--2025-03-19 08:23:07--  http://data.vision.ee.ethz.ch/cvl/DIV2K/DIV2K_valid_HR.zip
Resolving data.vision.ee.ethz.ch (data.vision.ee.ethz.ch)... 129.132.52.178, 2001:67c:10ec:36c2::178
Connecting to data.vision.ee.ethz.ch (data.vision.ee.ethz.ch)|129.132.52.178|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://data.vision.ee.ethz.ch/cvl/DIV2K/DIV2K_valid_HR.zip [following]
--2025-03-19 08:23:08--  https://data.vision.ee.ethz.ch/cvl/DIV2K/DIV2K_valid_HR.zip
Connecting to data.vision.ee.ethz.ch (data.vision.ee.ethz.ch)|129.132.52.178|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 448993893 (428M) [application/zip]
Saving to: ‘DIV2K_valid_HR.zip’

DIV2K_valid_HR.zip  100%[===================>] 428.19M  26.3MB/s    in 17s     

2025-03-19 08:23:25 (25.3 MB/s) - ‘DIV2K_valid_HR.zip’ saved [448993893/448993893]



In [11]:
import shutil
import os

zip_file = "//kaggle/working/noise.zip"
extract_folder = "/kaggle/working/noise"

shutil.unpack_archive(zip_file, extract_folder)
os.remove(zip_file)
print("Unzipped and deleted!")

Unzipped and deleted!


In [12]:
import shutil
import os

zip_file = "/kaggle/working/DIV2K_valid_HR.zip"
extract_folder = "/kaggle/working/DIV2K_valid_HR"

shutil.unpack_archive(zip_file, extract_folder)
os.remove(zip_file)
print("Unzipped and deleted!")

Unzipped and deleted!


In [13]:
class Val(Dataset):
    def __init__(self, clean_dir, noisy_dir, transform=None):
        self.clean_images = sorted(os.listdir(clean_dir))
        self.noisy_images = sorted(os.listdir(noisy_dir))
        self.clean_dir = clean_dir
        self.noisy_dir = noisy_dir
        self.transform = transform

    def __len__(self):
        return len(self.clean_images)

    def __getitem__(self, idx):
        clean_path = os.path.join(self.clean_dir, self.clean_images[idx])
        noisy_path = os.path.join(self.noisy_dir, self.noisy_images[idx])

        # Load images using OpenCV (NumPy arrays)
        clean_img = cv2.imread(clean_path)
        noisy_img = cv2.imread(noisy_path)

        # Convert BGR to RGB
        clean_img = cv2.cvtColor(clean_img, cv2.COLOR_BGR2RGB)
        noisy_img = cv2.cvtColor(noisy_img, cv2.COLOR_BGR2RGB)

        # Convert NumPy array to PIL Image
        clean_img = Image.fromarray(clean_img)
        noisy_img = Image.fromarray(noisy_img)

        # Apply transformations
        if self.transform:
            clean_img = self.transform(clean_img)
            noisy_img = self.transform(noisy_img)

        return noisy_img, clean_img

# Data augmentation & transformations
transform = transforms.Compose([
    transforms.Resize((512, 512)),  # Resize all images to 512x512
    transforms.ToTensor()  # Convert to PyTorch tensor
])

## Evaluation on Validation Dataset

In [14]:
# Define validation dataset paths
val_clean_dir = "/kaggle/working/DIV2K_valid_HR/DIV2K_valid_HR"
val_noisy_dir = "/kaggle/working/noise/noise"

# Create validation dataset and dataloader
val_dataset = Val(val_clean_dir, val_noisy_dir, transform=transform)
val_loader = DataLoader(val_dataset, batch_size=8, shuffle=False)

# Evaluate Model
evaluate_model(model, val_loader, device)

Validation PSNR: 28.23 dB, SSIM: 0.8022


(28.227648598152303, 0.8021559178829193)

In [17]:

# Save the trained model
model_save_path = "unet_model.pth"
torch.save(model.state_dict(), model_save_path)
print(f"Model saved to {model_save_path}")

Model saved to unet_model.pth
